In [1]:
pip install requests pandas


Note: you may need to restart the kernel to use updated packages.


In [ ]:
import requests
import pandas as pd
import time
import os

# RapidAPI credentials
RAPIDAPI_KEY = "8d4c87454fmsh34632789d84864ap1c1b12jsn9b500e1e3ebb"  # Replace with your API key
RAPIDAPI_HOST = "google-news13.p.rapidapi.com"

# API Endpoint
URL = "https://google-news13.p.rapidapi.com"

# Query Parameters (Modify as needed)
PARAMS = {
    "country": "us",  # Change for other countries (e.g., 'in', 'gb')
    "category": "technology",  # News category (e.g., 'sports', 'health')
}

# File to store news dataset
CSV_FILE = "real_time_news.csv"

def fetch_news():
    """Fetch latest news articles from the API."""
    headers = {
        "X-RapidAPI-Key": RAPIDAPI_KEY,
        "X-RapidAPI-Host": RAPIDAPI_HOST
    }
    
    response = requests.get(URL, headers=headers, params=PARAMS)
    
    if response.status_code == 200:
        return response.json().get("articles", [])
    else:
        print(f"Error fetching news: {response.status_code}")
        return []

def update_dataset():
    """Fetch latest news and update the dataset."""
    print("Fetching latest news...")
    articles = fetch_news()

    if not articles:
        print("No new articles found.")
        return
    
    # Convert API response to DataFrame
    df_new = pd.DataFrame(articles)[["title", "description", "url", "publishedAt", "source"]]
    df_new["source"] = df_new["source"].apply(lambda x: x["name"])  # Extract source name
    
    # Check if CSV file exists
    if os.path.exists(CSV_FILE):
        df_existing = pd.read_csv(CSV_FILE)
        
        # Avoid duplicates by removing already saved articles
        df_new = df_new[~df_new["title"].isin(df_existing["title"])]

        if df_new.empty:
            print("No new updates. Dataset is up to date.")
            return
        
        # Append new articles to existing dataset
        df_new.to_csv(CSV_FILE, mode='a', header=False, index=False)
    else:
        # Create new dataset file
        df_new.to_csv(CSV_FILE, index=False)

    print(f"Dataset updated! {len(df_new)} new articles added.")

# Run script continuously every 60 seconds
if __name__ == "__main__":
    while True:
        update_dataset()
        time.sleep(60)  # Wait 60 seconds before fetching again
